In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME


from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Put input_docs_sample.zip in your Google Drive

!rm -rf input_docs
!cp /content/drive/MyDrive/input_docs.zip .
!unzip input_docs.zip > /dev/null
!ls input_docs/ | wc -l

# for the real collection change above input_docs_sample.zip to input_docs.zip
# for the sample collection of 5 docs, the process is fast
# for the real collection, the process takes about 6 min (start to finish, the whole notebook) 

19026


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

**Create an RDD from a text file**

Each line of the text file becomes an element of the RDD.

In [ ]:
# wholeTextFiles generates an RDD of pair values, 
# where the key is the full path of each file, the value is the content of each file
#input = sc.wholeTextFiles("/content/drive/My\ Drive/input_docs");
input = sc.wholeTextFiles("input_docs");

# Now we strip the prefix of filenames and leave only the basename. 
# e.g. 'file:/content/drive/My Drive/Colab Notebooks/data_spark/input_docs/3.html'
# becomes '3.html' 
import os

#(did,text)
input2 = input.map(lambda x: (int(os.path.basename(x[0]).split(".")[0]), x[1]))

print(input2.take(1))

[(10932, "<H2>30-MAR-1987 13:07:39.17</H2>\r\n<H2>U.S. BILL SALES SEEN AT 5.74/72, 5.82/81 PCT</H2>\r\nThe U.S. Treasury's regular weekly\nbill auction is expected to produce an average yield of about\n5.74/72 pct for the three-month maturities and 5.82/81 pct for\nthe six-month, dealers said.\n    The current issues were yielding 5.71/70 and 5.76/75 pct\nrespectively in early afternoon trading.\n \n ")]


In [ ]:
# Doc to wordlist function
# The output will be a list of tuples such as 
# ("apple", (3,10,10/20)), 
# where 3 is docid, 
# 10 is frequency of "apple" in this doc, 
# 20 is maxf in in this doc.

from bs4 import BeautifulSoup
from collections import Counter
import re

from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

# for a given doc return a list of tuples of the form (w, (docid, freq, freq/maxfreq))
def dw(docid, htmltext):
  # Get required text
  cleantext = BeautifulSoup(htmltext).get_text()
  text = re.findall(r'(?:[^\W\d_]+\d|\d+[^\W\d_])[^\W_]*|[^\W\d_]+', cleantext.lower())
  result = []
  # remove stop words
  no_stop = [word for word in text if word not in stop_words]
  print(no_stop)
  count_freq = Counter(no_stop)
  # Find max frequency
  maxfreq = count_freq.most_common()[0][1]
  #print(maxfreq)
  for key in count_freq:
    result.append((key, (docid,count_freq[key],count_freq[key]/maxfreq)))  
  return result

word_docid_freq_tf = input2.flatMap(lambda x: dw(x[0],x[1]))

print(word_docid_freq_tf.take(2))

[('mar', (10932, 1, 0.25)), ('u', (10932, 2, 0.5))]


Expected output (all expected results are on the small sample):

<pre>
[('feb', (1, 1, 0.07142857142857142)), ('bahia', (1, 5, 0.35714285714285715))]
</pre>

In [ ]:
# Now create an RDD as follows 
# [(word, [(did1,freq1,tf1), (did2,freq2,tf2), ...(), ..., () ]


#TODO
word_postinglist_freq_tf = word_docid_freq_tf.groupByKey()

# creating a dummy RDD
#word_postinglist_freq_tf = sc.parallelize([ ('test', [(1, 1, 0.5), (2, 1, 0.2)]) ])

In [ ]:
# ( , []) result 
print(word_postinglist_freq_tf.map(lambda x : (x[0], list(x[1]))).take(1))

[('mar', [(10932, 1, 0.25), (4368, 1, 0.25), (4007, 1, 0.125), (10257, 1, 0.2), (3739, 1, 0.2), (1257, 1, 0.2), (3336, 1, 0.16666666666666666), (4854, 1, 0.25), (2022, 1, 0.25), (8389, 1, 0.2), (1390, 1, 0.25), (5539, 1, 0.125), (9636, 1, 0.04), (7110, 1, 0.25), (8170, 1, 0.25), (4051, 1, 0.2), (6696, 1, 0.3333333333333333), (2744, 1, 0.038461538461538464), (1380, 1, 0.1), (8497, 1, 0.1), (3279, 1, 0.25), (3846, 1, 0.3333333333333333), (10694, 1, 0.08333333333333333), (6695, 1, 0.5), (5345, 1, 0.047619047619047616), (1752, 1, 0.16666666666666666), (321, 1, 0.3333333333333333), (11307, 1, 0.14285714285714285), (6593, 1, 0.125), (2279, 1, 0.2), (4970, 1, 0.1), (3627, 1, 0.3333333333333333), (10132, 1, 0.3333333333333333), (5178, 1, 0.16666666666666666), (10347, 1, 0.1), (1826, 1, 0.3333333333333333), (10329, 1, 0.16666666666666666), (8614, 1, 0.058823529411764705), (8558, 1, 0.5), (1705, 1, 0.16666666666666666), (2879, 1, 0.25), (303, 1, 0.16666666666666666), (4074, 1, 0.1111111111111111

Expected output

<pre>
[('feb', [(1, 1, 0.07142857142857142), (2, 1, 0.2), (5, 1, 0.16666666666666666), (3, 1, 0.3333333333333333), (4, 1, 0.07142857142857142)]), ('bahia', [(1, 5, 0.35714285714285715)])]
</pre>

In [ ]:
# (word, [(did,freq,tfidf), ...])
# We easily obtain idf as 1/len(postinglist_tf)
# idf = 1/len(postinglist_tf)

#TODO
word_postinglist_freq_tfidf = word_postinglist_freq_tf.map(lambda x: (x[0], x[1],1/len(x[1]))) \
                                                           .map(lambda x: (x[0], [(dtuple[0],dtuple[1],dtuple[2]*x[2]) for dtuple in x[1]]))

# creating a dummy RDD
#word_postinglist_freq_tfidf = sc.parallelize([('test', [(1,3,0.12), (2,5,0.876)])])
print(word_postinglist_freq_tfidf.take(2))


[('mar', [(10932, 1, 2.372591819303407e-05), (4368, 1, 2.372591819303407e-05), (4007, 1, 1.1862959096517035e-05), (10257, 1, 1.8980734554427258e-05), (3739, 1, 1.8980734554427258e-05), (1257, 1, 1.8980734554427258e-05), (3336, 1, 1.5817278795356045e-05), (4854, 1, 2.372591819303407e-05), (2022, 1, 2.372591819303407e-05), (8389, 1, 1.8980734554427258e-05), (1390, 1, 2.372591819303407e-05), (5539, 1, 1.1862959096517035e-05), (9636, 1, 3.7961469108854515e-06), (7110, 1, 2.372591819303407e-05), (8170, 1, 2.372591819303407e-05), (4051, 1, 1.8980734554427258e-05), (6696, 1, 3.163455759071209e-05), (2744, 1, 3.65014126046678e-06), (1380, 1, 9.490367277213629e-06), (8497, 1, 9.490367277213629e-06), (3279, 1, 2.372591819303407e-05), (3846, 1, 3.163455759071209e-05), (10694, 1, 7.908639397678022e-06), (6695, 1, 4.745183638606814e-05), (5345, 1, 4.51922251295887e-06), (1752, 1, 1.5817278795356045e-05), (321, 1, 3.163455759071209e-05), (11307, 1, 1.3557667538876612e-05), (6593, 1, 1.18629590965170

Expected output

<pre>
[('feb', [(1, 1, 0.014285714285714285), (2, 1, 0.04), (5, 1, 0.03333333333333333), (3, 1, 0.06666666666666667), (4, 1, 0.014285714285714285)])]
</pre>

In [ ]:
# Now, we would like to obtain the magnitude of each doc.
# First, produce (did, (freq,tfidf)) for each word of doc did; 
# We do don't need the word itself, just its (freq,tfidf). 
# Then, do reduceByKey on these tuples and obtain maxfreq and 
# magnitude (squared) for each document. 

#TODO
did_freq_tfidfsq_rdd = word_postinglist_freq_tfidf.flatMap(lambda x: [tuple for tuple in x[1]]).map(lambda x: (x[0],(x[1],x[2]**2)))

print(did_freq_tfidfsq_rdd.take(2))

# Produce (did,(maxf,magnitudesq))

doc_maxf_mag = did_freq_tfidfsq_rdd.reduceByKey(lambda a, b: (max(a[0],b[0]),a[1]+b[1] ))

print(doc_maxf_mag.take(2))

[(10932, (1, 5.629191941025451e-10)), (4368, (1, 5.629191941025451e-10))]
[(10932, (4, 3.0439520486657982e-05)), (4368, (4, 0.3168712497652501))]


Excpected result

<pre>
[(1, (1, 0.0002040816326530612)), (2, (1, 0.0016))]
[(2, (5, 3.894100000000001)), (4, (14, 2.94553429705215))]
</pre>

In [ ]:
!rm -rf inv_idx
word_postinglist_freq_tfidf.saveAsTextFile("inv_idx");

In [ ]:
!rm -rf doc_mag
doc_maxf_mag.saveAsTextFile("doc_mag");

In [ ]:
!ls -lrt inv_idx
!head inv_idx/part-00001
!wc -l inv_idx/part-00000
!wc -l inv_idx/part-00001
!cat inv_idx/part-00000 inv_idx/part-00001 > /content/drive/MyDrive/inv_idx.txt
!wc -l /content/drive/MyDrive/inv_idx.txt

total 37844
-rw-r--r-- 1 root root 17980560 Apr 13 00:29 part-00001
-rw-r--r-- 1 root root 20768174 Apr 13 00:29 part-00000
-rw-r--r-- 1 root root        0 Apr 13 00:29 _SUCCESS
('u', [(10932, 2, 0.00011111111111111112), (1257, 1, 4.4444444444444447e-05), (2022, 1, 5.555555555555556e-05), (8389, 1, 4.4444444444444447e-05), (14872, 1, 2.2222222222222223e-05), (5539, 3, 8.333333333333334e-05), (9636, 5, 4.4444444444444447e-05), (18177, 1, 3.1746031746031745e-05), (8497, 1, 2.2222222222222223e-05), (21512, 2, 6.349206349206349e-05), (19409, 1, 4.4444444444444447e-05), (19139, 1, 2.4691358024691357e-05), (5345, 2, 2.1164021164021164e-05), (321, 2, 0.00014814814814814815), (16153, 6, 0.00022222222222222223), (14879, 2, 7.407407407407407e-05), (14779, 1, 7.407407407407407e-05), (2879, 1, 5.555555555555556e-05), (303, 2, 7.407407407407407e-05), (13772, 3, 8.333333333333334e-05), (11020, 1, 4.4444444444444447e-05), (12883, 1, 3.7037037037037037e-05), (17363, 10, 0.0001388888888888889), (18941,

In [ ]:
!ls -lrt doc_mag
!head doc_mag/part-00000
!wc -l doc_mag/part-00000
!wc -l doc_mag/part-00001
!cat doc_mag/part-00000 doc_mag/part-00001 > /content/drive/MyDrive/doc_mag.txt
!wc -l /content/drive/MyDrive/doc_mag.txt

total 636
-rw-r--r-- 1 root root 324125 Apr 13 00:29 part-00001
-rw-r--r-- 1 root root 323080 Apr 13 00:29 part-00000
-rw-r--r-- 1 root root      0 Apr 13 00:29 _SUCCESS
(10932, (4, 3.0439520486657982e-05))
(4368, (4, 0.3168712497652501))
(3336, (6, 0.007630045748983177))
(4854, (4, 0.24339230773407852))
(2022, (4, 0.0026182921622585507))
(1390, (4, 0.002347289450311327))
(9636, (25, 0.004902510081880886))
(7110, (4, 0.3060748329876748))
(8170, (4, 0.25699276008988403))
(6696, (3, 0.01234667582824942))
9500 doc_mag/part-00000
9526 doc_mag/part-00001
19026 /content/drive/MyDrive/doc_mag.txt


### Next Step
2. Create database tables for storing the inverted index.
3. Implement the keyword search functionality.
4. Implement result ranking using the TF-IDF measure.
5. Implement a simple interface for giving keyword queries and showing results.

In [ ]:
# Part 2: Create database tables for storing the inverted index
import sqlite3

# create a Connection object that represents the database
conn = sqlite3.connect('data.db')

# create a Cursor object 
curs = conn.cursor()

curs.execute('''DROP TABLE IF EXISTS postings''')
curs.execute('''CREATE TABLE IF NOT EXISTS postings (
            word VARCHAR(100) PRIMARY KEY,
            postinglist_freq_tfidf TEXT
            );''')
print("Table schema postings created.")

curs.execute('''DROP TABLE IF EXISTS docmag''')
curs.execute('''CREATE TABLE IF NOT EXISTS docmag (
            docid INT PRIMARY KEY,
            maxf INT,
            mag FLOAT
            );''')
print("Table schema docmag created.")

conn.commit()

Table schema postings created.
Table schema docmag created.


In [ ]:
# Store inv_idx data to the table invidx
curs.execute('''DELETE FROM postings''')
with open("/content/drive/My Drive/inv_idx.txt") as f1:
  ii_content = f1.readlines()

for i in ii_content:
  ti = re.findall(r'\'(.*)\'[^\[]*(\[.*\])', i)
  #print(ti[0][1])
  curs.execute('''INSERT INTO postings VALUES (?,?)''', [ti[0][0], ti[0][1]])
  conn.commit()

In [ ]:
# Store doc_mag data to the table docmag
curs.execute('''DELETE FROM docmag''')
with open("/content/drive/My Drive/doc_mag.txt") as f2:
  dm_content = f2.readlines()

for j in dm_content:
  tm = re.findall(r'\d+\.?\d*', j)
  #print(tm[1])
  curs.execute('''INSERT INTO docmag VALUES (?,?,?)''', [str(tm[0]), tm[1], tm[2]])
  conn.commit()

In [ ]:
# Search Functionality
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from collections import Counter
import math
import operator

# in posting: (word, [(did,freq,tfidf), ...])
# in docmag: (did,(maxf,magnitudesq))
# Notes: Term Frequency (TF) Scheme, The entries will be (i, [(j1,fij)])
# tfij = fij / maxfj
# idfi = log(N/dfi)
# wij= tfij * idfi
# N: total number of docs
# dfi: the number of docs that ti appears. 

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def search_fun(word_in):
  keyword = word_in
  keyword = str(keyword).lower()
  cleantext = BeautifulSoup(keyword).get_text()
  text = re.findall(r'(?:[^\W\d_]+\d|\d+[^\W\d_])[^\W_]*|[^\W\d_]+', cleantext.lower())
  # remove stop words
  no_stop = [word for word in text if word not in stop_words]
  count_freq = Counter(no_stop)
  # Find max frequency
  maxfreq = count_freq.most_common(1)[0][1]

  # Search in postings, may have multiple tuples 
  sql_query='''SELECT * FROM postings WHERE word IN ({seq})'''.format(seq=','.join(['?']*len(no_stop)))
  curs.execute(sql_query, no_stop)
  search_result = curs.fetchall()
  
  # remove the word and get doc id from the search result
  sr_no_word = dict()
  for j in search_result:
    docid = re.findall(r'\((.*?),.*?[+-]?([0-9]*[.]?[0-9]+)\)', j[1])
    for z in docid:
      sr_no_word[str(z[0])] = {}
      sr_no_word[str(z[0])][str(j[0])] = z[1]
  unique_docid = list(sr_no_word.keys())

  # Use the unique id to search in docmag
  sr_mag = dict()
  for a in unique_docid:
    curs.execute('''SELECT * FROM docmag WHERE docid = ?''', [a])
    mag_result = curs.fetchall()
    # save each doc id and corresponding maxf in sr_maxf as a dictionary
    # push query mag in sr_mag
    for k in mag_result:
      sr_mag[str(k[0])] = k[2] 

  # Cosine Similarity Calculation, score = (q*d) / (|q|*|d|)
  query = dict()
  d_norm = 0   # this is the doc magnitude = |d|
  for key in count_freq:
    tf = count_freq[key]/maxfreq
    query[key] = tf
    d_norm = d_norm + tf**2
  d_norm = math.sqrt(d_norm)

  cosine_sim = dict()
  for id in unique_docid:
    numerator = 0
    for t in no_stop:
      d = float(sr_no_word[id][t])
      q = float(query.get(t))
      numerator += d * q
      denominator = sr_mag.get(id) * d_norm
      cosine_sim[id] = numerator/denominator
  #print(cosine_sim)

  # return: sorted list starting with the highest and ending with the lowest
  rank_result = sorted(cosine_sim.items(), key=operator.itemgetter(1),reverse=True)
  print("Search result: \nRelevent document id displayed from highest similarity to lowest similarity.")
  for item in rank_result:
    print(item[0])

def on_button_clicked(b):
    with output:
        search_fun(keyword.value)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Part 3: Implement the keyword search functionality.
# Part 4: Implement result ranking using the TF-IDF measure.
# Part 5: Implement a simple interface for giving keyword queries and showing results.

# Search Interface
import ipywidgets as widgets

keyword = widgets.Text(placeholder='Enter the keyword')
button = widgets.Button(description="Search")
output = widgets.Output()
display(keyword, button, output)
button.on_click(on_button_clicked)

Text(value='', placeholder='Enter the keyword')

Button(description='Search', style=ButtonStyle())

Output()